In [24]:
# This is only to fix autocomplete in my version of Jupyter notebook.  Disregard if you don't have this problem
# %config Completer.use_jedi = False

# This is to allow the use pyqtgraph for plots
%gui qt

In [25]:
from easy_wave import Block_Writer, AWG_Writer, Channel, to_integer, PulseLengthError
from wave_library import *
from visual_wave import plot_line, plot_block
from tqdm import tqdm_notebook

from easy_wave import AWG_Writer, Channel, REAL_CHANNELS
from wave_library import *
from visual_wave import plot_line

In [26]:
# First let's give names to our channels
chs = {
    'sg1_i' : Channel.ch1_a,
    'sg1_g' : Channel.ch1_m1,
    'offres': Channel.ch1_m2,
    
    'sg1_q'  : Channel.ch2_a,
    'lockin' : Channel.ch2_m1,
    'readout': Channel.ch2_m2,
    
    'pico_trig': Channel.ch3_a,
    'red'      : Channel.ch3_m1,
    'rocket'   : Channel.ch3_m2,
    
    'res_analog': Channel.ch4_a,
    'solar'     : Channel.ch4_m1,
    'hyper'     : Channel.ch4_m2,
}
sg1_chs = [chs[x] for x in ['sg1_i','sg1_q', 'sg1_g']]

shifts = {
            chs['solar']: -470e-9,
            chs['offres']: -390e-9,
        }

working_dir = r"2019-08 Isotopically Purified/"

In [27]:
def basic_pulses(mw_pi = 100e-9, square_Ahpi=0.8888, init_time = 200e-6, readout_time = 100e-6,
                 delay_time = 2e-6, init_laser = 'offres', read_laser = 'hyper', mw_buffer=50e-9):
    """These are a basic set of init, read and square mw pulses"""
    init = Rect(t=init_time, ch=chs[init_laser])
    readout  = Rect(t=readout_time, ch=chs[read_laser])
    readout &= Rect(t=readout_time, ch=chs['readout'])
    delay = Zero(t=delay_time, ch=Channel.no_ch)
    wait = lambda _t: Zero(t=_t, ch=Channel.no_ch)
    mw = lambda t, phase, A: IQ_MW_Pulse(t, *sg1_chs, phase=phase, A=A, freq_shift=0, mw_buffer=mw_buffer)
    pi = lambda phase: mw(mw_pi, phase, 1)
    hpi = lambda phase: mw(mw_pi/2, phase, square_Ahpi)
    return init, readout, delay, wait, mw, pi, hpi  

In [28]:
def gen(writer=AWG_Writer(shifts=shifts)):
    pi_time = 100e-9
    init, readout, delay, wait, mw, pi, hpi = basic_pulses(mw_pi=pi_time)
    N, factor = 10, 10
    for i in tqdm_notebook(range(N)):
        number_of_pi = i*factor + 1
        seq = (pi(0) + wait(pi_time))*number_of_pi
        buffer = (pi(0).blank() + wait(pi_time))*(N*factor-number_of_pi)
        line  = delay + init + delay + seq         + buffer + readout
        line += delay + init + delay + seq.blank() + buffer + readout
        line &= RectCycle(t=line.t, ch=chs['lockin'])
        writer.add_line(line, 'Test i={} (pi={:.0f}ns)'.format(number_of_pi, pi_time/ 1e-9))
    return writer
writer = gen().generate_and_upload('192.168.1.115', 'temp/t1test2.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

<ipython-input-28-cd4044107308>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N)):


  0%|          | 0/10 [00:00<?, ?it/s]

file generated


EOFError: 

### T1 code with specified max and min delay times

In [23]:
def basic_pulses(init_time = 200e-6, readout_time = 100e-6, init_laser = 'offres', read_laser = 'hyper'):
    init = Rect(t=init_time, ch=chs[init_laser])
    readout  = Rect(t=readout_time, ch=chs[read_laser])
    readout &= Rect(t=readout_time, ch=chs['readout'])  
    return init, readout

def t1(writer=AWG_Writer(shifts=shifts), N=100, min_delay = (0.22e-3)/5, max_delay = (0.22e-3)*5):
    init, readout = basic_pulses()
    factor = (max_delay - min_delay)/(N-1)
    # round factor to the 8th decimal point so that the length of the delays
    # are integer multiples of the AWG's time step
    factor = round(factor,5) 
    
    print("the interpulse spacing increases linearly by " + str(factor) + " seconds from line-to-line")
    for i in tqdm_notebook(range(N)):
        # total time of sequence should always stay the same
        # increasing_delay_time + decreasing_delay_time = max_delay
        # Therefore total experiment time will be init_time*2 + max_delay + read_time*2
        increasing_delay_time = i*factor + min_delay
        decreasing_delay_time = (N-1)*factor + min_delay - increasing_delay_time
        increasing_delay= Zero(t=increasing_delay_time, ch=chs['offres'])
        decreasing_delay = Zero(t=decreasing_delay_time, ch=chs['offres'])
        line = init + increasing_delay + readout
        line += init + decreasing_delay + readout
        #print(increasing_delay_time)
        writer.add_line(line, 'Test i={} (idt={:.0f})'.format(i, increasing_delay_time))
    return writer

writer = t1().generate_and_upload('192.168.0.144', 'temp/t1.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

the interpulse spacing increases linearly by 1e-05 seconds from line-to-line


<ipython-input-23-e43efeb19727>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N)):


4.4e-05
5.4e-05
6.4e-05
7.400000000000001e-05
8.400000000000001e-05
9.400000000000001e-05
0.00010400000000000001
0.000114
0.000124
0.000134
0.000144
0.000154
0.00016400000000000003
0.000174
0.00018400000000000003
0.000194
0.00020400000000000003
0.000214
0.00022400000000000002
0.000234
0.00024400000000000002
0.000254
0.000264
0.000274
0.000284
0.000294
0.000304
0.000314
0.000324
0.000334
0.000344
0.000354
0.000364
0.00037400000000000004
0.000384
0.00039400000000000004
0.000404
0.00041400000000000003
0.000424
0.00043400000000000003
0.000444
0.00045400000000000003
0.000464
0.00047400000000000003
0.000484
0.0004940000000000001
0.000504
0.000514
0.000524
0.0005340000000000001
0.000544
0.000554
0.000564
0.0005740000000000001
0.000584
0.000594
0.000604
0.0006140000000000001
0.000624
0.000634
0.000644
0.0006540000000000001
0.000664
0.000674
0.000684
0.0006940000000000001
0.0007040000000000001
0.000714
0.000724
0.0007340000000000001
0.0007440000000000001
0.000754
0.000764
0.0007740000000000001


### T1 code with specified time increase

In [32]:
def basic_pulses(init_time = 200e-6, readout_time = 100e-6, init_laser = 'offres', read_laser = 'hyper'):
    init = Rect(t=init_time, ch=chs[init_laser])
    readout  = Rect(t=readout_time, ch=chs[read_laser])
    readout &= Rect(t=readout_time, ch=chs['readout'])  
    return init, readout

def t1(writer=AWG_Writer(shifts=shifts), N=100, min_delay = (0.22e-3)/5):
    init, readout = basic_pulses()
    factor = 0.01e-3 #the interpulse spacing increases linearly by *factor* seconds from line-to-line
    for i in tqdm_notebook(range(N)):
        # total time of sequence should always stay the same
        # increasing_delay_time + decreasing_delay_time = max_delay
        # Therefore total experiment time will be init_time*2 + max_delay + read_time*2
        increasing_delay_time = i*factor + min_delay
        decreasing_delay_time = (N-1)*factor + min_delay - increasing_delay_time
        increasing_delay= Zero(t=increasing_delay_time, ch=chs['offres'])
        decreasing_delay = Zero(t=decreasing_delay_time, ch=chs['offres'])
        line = init + increasing_delay + readout
        line += init + decreasing_delay + readout
        #print(increasing_delay_time)
        writer.add_line(line, 'Test i={} (idt={:.0f})'.format(i, increasing_delay_time))
    return writer

writer = t1().generate_and_upload('192.168.1.115', 'temp/t1.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

<ipython-input-32-3e28dc7ba197>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N)):



file generated
opening ftp at 192.168.0.144
uploaded


<ipython-input-32-3e28dc7ba197>:25: RuntimeWarning: Visible window deleted. To prevent this, store a reference to the window object.
  plt = plot_line(writer, 10)


### T1 code (with delays in sequence)

In [47]:
def basic_pulses(init_time = 200e-6, readout_time = 100e-6, init_laser = 'offres', read_laser = 'offres'):
    init = Rect(t=init_time, ch=chs[init_laser])
    readout  = Rect(t=readout_time, ch=chs[read_laser])
    readout &= Rect(t=readout_time, ch=chs['readout'])  
    return init, readout

def t1(writer=AWG_Writer(shifts=shifts), N=100, min_delay = (0.22e-3)/5):
    init, readout = basic_pulses()
    factor = 0.01e-3 #the interpulse spacing increases linearly by *factor* seconds from line-to-line
    for i in tqdm_notebook(range(N)):
        # total time of sequence should always stay the same
        # increasing_delay_time + decreasing_delay_time = max_delay
        # Therefore total experiment time will be init_time*2 + max_delay + read_time*2
        increasing_delay_time = i*factor + min_delay
        decreasing_delay_time = (N-1)*factor + min_delay - increasing_delay_time
        increasing_delay= Zero(t=increasing_delay_time, ch=chs['offres'])
        decreasing_delay = Zero(t=decreasing_delay_time, ch=chs['offres'])
        line = init + increasing_delay + readout + decreasing_delay
        #print(increasing_delay_time)
        writer.add_line(line, 'Test i={} (idt={:.0f})'.format(i, increasing_delay_time))
    return writer

writer = t1().generate_and_upload('192.168.1.115', 'temp/t1_one_measurement_per_line.awg', rate=1e8, limits={})
plt = plot_line(writer, 10)

<ipython-input-47-bd2e80dfd159>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N)):


  0%|          | 0/100 [00:00<?, ?it/s]

file generated
opening ftp at 192.168.1.115
uploaded


In [15]:
t1().getline()

<ipython-input-8-0d758dfc4688>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(N)):


  0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'AWG_Writer' object has no attribute 'getline'

In [44]:
# This is only to fix autocomplete in my version of Jupyter notebook.  Disregard if you don't have this problem
# %config Completer.use_jedi = False

# This is to allow the use pyqtgraph for plots
%gui qt

In [45]:
from easy_wave import AWG_Writer, Channel, REAL_CHANNELS
from wave_library import *
from visual_wave import plot_line

# First let's give names to our channels
chs = {
    'sg1_i' : Channel.ch1_a,
    'sg1_g' : Channel.ch1_m1,
    'offres': Channel.ch1_m2,
    
    'sg1_q'  : Channel.ch2_a,
    'lockin' : Channel.ch2_m1,
    'readout': Channel.ch2_m2,
    
    'pico_trig': Channel.ch3_a,
    'red'      : Channel.ch3_m1,
    'rocket'   : Channel.ch3_m2,
    
    'res_analog': Channel.ch4_a,
    'solar'     : Channel.ch4_m1,
    'hyper'     : Channel.ch4_m2,
}
sg1_chs = [chs[x] for x in ['sg1_i','sg1_q', 'sg1_g']]

In [46]:
# Creating some basic pulses
mw = IQ_MW_Pulse(0.1e-6, *sg1_chs, freq_shift=100e6)
init = Rect(t=2e-6, ch=chs['offres'])
readout = Rect(t=3e-6, ch=chs['hyper'])
delay = Zero(t=1e-6, ch=chs['offres'])

#Create the sequence
line = init + delay + mw + delay + readout + delay

# Use AWG_Writer to write a file and send it via ftp
writer = AWG_Writer()
writer.add_line(line, 'test')
writer.add_line(line, 'test')
writer.generate_and_upload('192.168.1.115', 'temp/test2.awg')


file generated
opening ftp at 192.168.1.115
uploaded
